# Using our ridehail API for on-demand goods delivery

This jupyter notebook will run you through a simple on-demand goods delivery application of rideOS' ridehail API. Here is what we're going to do:

1. We create a fleet and add a vehicle to the fleet. 
2. We update the state (position and readiness) of the vehicle. 
3. We request 2 deliveries and complete all the steps that happen throughout both trips.
4. We clean up the vehicle and the fleet. 

In [1]:
!pip install requests

import requests
import json
import uuid

## API Key

To run this example, you'll need a rideOS API key. You can sign up for one [here](https://app.rideos.ai/login#lockScreen=signUp) and view it on your [profile page](https://app.rideos.ai/profile), then assign it to the `API_KEY` variable below:


In [2]:
# IMPORTANT: replace "YOUR_RIDEOS_API_KEY" with your actual rideOS API key
API_KEY = "YOUR_RIDEOS_API_KEY"

AUTHORIZATION_HEADER = {"X-Api-Key": API_KEY}

## Creating UUIDs

Creating fleet, vehicle and rider uuids using uuid4.

In [3]:
fleet_id = str(uuid.uuid4())
vehicle_id = str(uuid.uuid4())

print("Fleet id: " + fleet_id)
print("Vehicle id: " + vehicle_id)

Fleet id: d30f8cbf-c1ce-4f77-b01a-7bc19d6114c5
Vehicle id: 5a1a4868-4eb7-42eb-a158-63a81205f356


## ride-hail-operations: CreateFleet

Creating a fleet.

In [4]:
CREATE_FLEET_URL = "https://api.rideos.ai/ride-hail-operations/v1/CreateFleet"

create_fleet_request = {
    "id": fleet_id,
    "info": {
        "display_name": "Sample On-Demand Goods Delivery Fleet"
    }
}

response = requests.post(
    CREATE_FLEET_URL,
    headers=AUTHORIZATION_HEADER,
    json=create_fleet_request
)
response.raise_for_status()

create_fleet_response = response.json()

print("Got successful create fleet response:")
print(json.dumps(create_fleet_response, indent=4))


Got successful create fleet response:
{}


## ride-hail-operations: GetFleets

Getting the fleet to make sure we've created the right fleet.

In [5]:
GET_FLEET_URL = "https://api.rideos.ai/ride-hail-operations/v1/GetFleets"

get_fleet_request = {}

response = requests.post(
    GET_FLEET_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_fleet_request
)
response.raise_for_status()

get_fleet_response = response.json()

print("Got successful get fleet response:")
print(json.dumps(get_fleet_response, indent=4))

Got successful get fleet response:
{
    "fleet": [
        {
            "id": "",
            "info": {
                "displayName": "Default fleet",
                "properties": {}
            },
            "optimizationSettings": {}
        },
        {
            "id": "d30f8cbf-c1ce-4f77-b01a-7bc19d6114c5",
            "info": {
                "displayName": "Sample On-Demand Goods Delivery Fleet",
                "properties": {}
            },
            "optimizationSettings": {
                "forcePrivateRides": false,
                "enableVehicleReassignment": false
            }
        },
        {
            "id": "sim_fleet_caed0c",
            "info": {
                "displayName": "Web Sim Fleet",
                "properties": {
                    "is_simulated": "true"
                }
            },
            "optimizationSettings": {
                "forcePrivateRides": false,
                "enableVehicleReassignment": false
            }
       

## ride-hail-driver: CreateVehicle

Adding a vehicle to a fleet using ride-hail-driver. Each vehicle can only belong to one fleet.

In [6]:
CREATE_VEHICLE_URL = "https://api.rideos.ai/ride-hail-driver/v1/CreateVehicle"

create_vehicle_request = {
    "id": vehicle_id,
    "fleet_id": fleet_id,
    "definition": {
        "rider_capacity": 4
    },
    "info": {
        "driver_info": {
            "contact_info": {
                "name": "driver1",
                "phone_number": 987654321
            }
        },
        "properties": {
            "make": "Toyota",
            "model": "Camry"
        }
    }
}
    
response = requests.post(
    CREATE_VEHICLE_URL,
    headers=AUTHORIZATION_HEADER,
    json=create_vehicle_request
)
response.raise_for_status()

create_vehicle_response = response.json()

print("Got successful create vehicle response:")
print(json.dumps(create_vehicle_response, indent=4))

Got successful create vehicle response:
{}


## ride-hail-operations: GetVehicles

Get the vehicles in the fleet.

In [7]:
GET_VEHICLES_URL = "https://api.rideos.ai/ride-hail-operations/v1/GetVehicles"

get_vehicles_request = {
    "fleet_id": fleet_id
}

response = requests.post(
    GET_VEHICLES_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicles_request
)
response.raise_for_status()

get_vehicles_response = response.json()

print("Got successful get vehicles response:")
print(json.dumps(get_vehicles_response, indent=4))

Got successful get vehicles response:
{
    "vehicle": [
        {
            "id": "5a1a4868-4eb7-42eb-a158-63a81205f356",
            "fleetId": "d30f8cbf-c1ce-4f77-b01a-7bc19d6114c5",
            "definition": {
                "riderCapacity": 4
            },
            "info": {
                "driverInfo": {
                    "contactInfo": {
                        "name": "driver1",
                        "phoneNumber": "987654321",
                        "contactUrl": ""
                    }
                },
                "properties": {
                    "make": "Toyota",
                    "model": "Camry"
                },
                "licensePlate": ""
            },
            "state": {
                "plan": {
                    "step": []
                },
                "readiness": false,
                "isReachable": false
            }
        }
    ]
}


## ride-hail-driver: UpdateVehicleState - UpdatePosition

Setting the vehicle's current position by updating the vehicle state.

In [8]:
UPDATE_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/UpdateVehicleState"

update_vehicle_state_request = {
    "id": vehicle_id,
    "update_position": {
        "updated_position": { # San Francisco
            "latitude": 37.788897, 
            "longitude": -122.389669
        },
        "updated_heading": 0.0
    }
}
    
response = requests.post(
    UPDATE_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=update_vehicle_state_request
)
response.raise_for_status()

update_vehicle_state_response = response.json()

print("Got successful update vehicle state response:")
print(json.dumps(update_vehicle_state_response, indent=4))

Got successful update vehicle state response:
{}


## ride-hail-driver: UpdateVehicleState - SetToAcceptRides

Marking the vehicle to accept rides by updating the vehicle state.

In [9]:
UPDATE_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/UpdateVehicleState"

update_vehicle_state_request = {
    "id": vehicle_id,
    "set_to_accept_rides": {}
}
    
response = requests.post(
    UPDATE_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=update_vehicle_state_request
)
response.raise_for_status()

update_vehicle_state_response = response.json()

print("Got successful update vehicle state response:")
print(json.dumps(update_vehicle_state_response, indent=4))

Got successful update vehicle state response:
{}


## ride-hail-driver: GetVehicleState

Checking the vehicle state. The plan is empty since the vehicle has no assigned trips yet.

In [10]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}
    
response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

print("Got successful get vehicle state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get vehicle state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": []
        },
        "readiness": true,
        "isReachable": true
    }
}


## ride-hail-rider: RequestTrip

Requesting delivery A using ride-hail-rider with a pickup, dropoff and 2 packages.

In [11]:
REQUEST_TRIP_URL = "https://api.rideos.ai/ride-hail-rider/v1/RequestTrip"

delivery_id_a = str(uuid.uuid4())
rider_id_a = str(uuid.uuid4())

print("Delivery id A: " + delivery_id_a)
print("Rider id A: " + rider_id_a)

request_trip_request = {
    "id": delivery_id_a,
    "rider_id": rider_id_a,
    "fleet_id": fleet_id,
    "definition": {
        "pickup_dropoff": {
            "pickup": { # rideOS HQ
                "position": {
                    "latitude": 37.790329, 
                    "longitude": -122.392231
                }
            },
            "dropoff": { # Union Square
                "position": {
                    "latitude": 37.787481, 
                    "longitude": -122.407543
                }
            },
            "rider_count": 2
        }
    },
    "info": {
        "rider_info": {
            "contact_info": {
                "name": "sample app rider1",
                "phone_number": 123456789
            }
        }
    }
}

response = requests.post(
    REQUEST_TRIP_URL,
    headers=AUTHORIZATION_HEADER,
    json=request_trip_request
)
response.raise_for_status()

request_trip_response = response.json()

print("Got successful request trip response:")
print(json.dumps(request_trip_response, indent=4))

Delivery id A: fa28a144-4f6b-422c-b2bc-2b2fed67e8d1
Rider id A: d79e1262-0ef6-4d3a-815d-e849c8f0db6e
Got successful request trip response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. If you scroll through the plan of the vehicle, you'll notice the plan looks like drive, pickup, drive, dropoff.

Something to note: if GetVehicleState is called really quickly after RequestTrip, we might not have a vehicle assigned yet. In that situation, wait a couple seconds and re-run the cell. 

In [12]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "47dc7637-bee8-4ff1-b26d-28363563211a",
                    "tripId": "fa28a144-4f6b-422c-b2bc-2b2fed67e8d1",
                    "location": {
                        "position": {
                            "latitude": 37.790329,
                            "longitude": -122.392231
                        }
                    },
                    "position": {
                        "latitude": 37.790329,
                        "longitude": -122.392231
                    },
                    "driveToLocation": {}
                },
                {
                    "id": "e8a33cf9-45bb-4a53-9045-f9f069b1235d",
                    "tripId

## ride-hail-rider: RequestTrip

Requesting delivery B using ride-hail-rider with a pickup, dropoff and 2 packages.

In [13]:
REQUEST_TRIP_URL = "https://api.rideos.ai/ride-hail-rider/v1/RequestTrip"

delivery_id_b = str(uuid.uuid4())
rider_id_b = str(uuid.uuid4())

print("Delivery id B: " + delivery_id_b)
print("Rider id B: " + rider_id_b)

request_trip_request = {
    "id": delivery_id_b,
    "rider_id": rider_id_b,
    "fleet_id": fleet_id,
    "definition": {
        "pickup_dropoff": {
            "pickup": { # Philz 
                "position": {
                    "latitude": 37.788667, 
                    "longitude": -122.393168
                }
            },
            "dropoff": { # Fisherman's Wharf
                "position": {
                    "latitude": 37.808410, 
                    "longitude": -122.410097
                }
            },
            "rider_count": 2
        }
    },
    "info": {
        "rider_info": {
            "contact_info": {
                "name": "sample app rider1",
                "phone_number": 123456789
            }
        }
    }
}

response = requests.post(
    REQUEST_TRIP_URL,
    headers=AUTHORIZATION_HEADER,
    json=request_trip_request
)
response.raise_for_status()

request_trip_response = response.json()

print("Got successful request trip response:")
print(json.dumps(request_trip_response, indent=4))

Delivery id B: 9500ca86-eb47-47e9-8c6b-ea52b9d70190
Rider id B: f1b0aaa4-7d9e-4266-847a-a26bfb09185b
Got successful request trip response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. There should now be 2 trips assigned to the vehicle. If you scroll through the plan of the vehicle, you'll notice that there are 2 trip ids that are assigned. The plan will look like drive, pickup, drive, pickup, drive, dropoff, drive, dropoff.

In [14]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "47dc7637-bee8-4ff1-b26d-28363563211a",
                    "tripId": "fa28a144-4f6b-422c-b2bc-2b2fed67e8d1",
                    "location": {
                        "position": {
                            "latitude": 37.790329,
                            "longitude": -122.392231
                        }
                    },
                    "position": {
                        "latitude": 37.790329,
                        "longitude": -122.392231
                    },
                    "driveToLocation": {}
                },
                {
                    "id": "e8a33cf9-45bb-4a53-9045-f9f069b1235d",
                    "tripId

## ride-hail-driver: CompleteSteps

Driving to first pickup.

In [15]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now look like pickup, drive, pickup, drive, dropoff, drive, dropoff since the drive to first pickup step has been completed.

In [16]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "e8a33cf9-45bb-4a53-9045-f9f069b1235d",
                    "tripId": "fa28a144-4f6b-422c-b2bc-2b2fed67e8d1",
                    "location": {
                        "position": {
                            "latitude": 37.790329,
                            "longitude": -122.392231
                        }
                    },
                    "position": {
                        "latitude": 37.790329,
                        "longitude": -122.392231
                    },
                    "pickupRider": {
                        "riderCount": 2,
                        "riderInfo": {
                            "contactInfo": {
          

## ride-hail-driver: CompleteSteps

First pickup.

In [17]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now look like drive, pickup, drive, dropoff, drive, dropoff since the first pickup step has been completed.

In [18]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "1c7e17d0-e370-42eb-b3b8-e4735cd707ae",
                    "tripId": "9500ca86-eb47-47e9-8c6b-ea52b9d70190",
                    "location": {
                        "position": {
                            "latitude": 37.788667,
                            "longitude": -122.393168
                        }
                    },
                    "position": {
                        "latitude": 37.788667,
                        "longitude": -122.393168
                    },
                    "driveToLocation": {}
                },
                {
                    "id": "d1c9be43-a93b-457f-93fb-7dd62b920de7",
                    "tripId

## ride-hail-driver: CompleteSteps

Driving to second pickup.

In [19]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now look like pickup, drive, dropoff, drive, dropoff since the drive to second pickup step has been completed.

In [20]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "d1c9be43-a93b-457f-93fb-7dd62b920de7",
                    "tripId": "9500ca86-eb47-47e9-8c6b-ea52b9d70190",
                    "location": {
                        "position": {
                            "latitude": 37.788667,
                            "longitude": -122.393168
                        }
                    },
                    "position": {
                        "latitude": 37.788667,
                        "longitude": -122.393168
                    },
                    "pickupRider": {
                        "riderCount": 2,
                        "riderInfo": {
                            "contactInfo": {
          

## ride-hail-driver: CompleteSteps

Second pickup.

In [21]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now look like drive, dropoff, drive, dropoff since the second pickup step has been completed.

In [22]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "a856eaea-a820-43a2-acf1-21197e649b76",
                    "tripId": "fa28a144-4f6b-422c-b2bc-2b2fed67e8d1",
                    "location": {
                        "position": {
                            "latitude": 37.787481,
                            "longitude": -122.407543
                        }
                    },
                    "position": {
                        "latitude": 37.787481,
                        "longitude": -122.407543
                    },
                    "driveToLocation": {}
                },
                {
                    "id": "0c39e7b8-720b-4e17-bfef-efb3824dc6c9",
                    "tripId

## ride-hail-driver: CompleteSteps

Driving to first dropoff.

In [23]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now look like dropoff, drive, dropoff since the drive to first dropoff step has been completed.

In [24]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "0c39e7b8-720b-4e17-bfef-efb3824dc6c9",
                    "tripId": "fa28a144-4f6b-422c-b2bc-2b2fed67e8d1",
                    "location": {
                        "position": {
                            "latitude": 37.787481,
                            "longitude": -122.407543
                        }
                    },
                    "position": {
                        "latitude": 37.787481,
                        "longitude": -122.407543
                    },
                    "dropoffRider": {
                        "riderCount": 2,
                        "riderInfo": {
                            "contactInfo": {
         

## ride-hail-driver: CompleteSteps

First dropoff.

In [25]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now look like drive, dropoff since the first dropoff step has been completed.

In [26]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "4deb7f23-c639-4a63-a3bd-307ab5183003",
                    "tripId": "9500ca86-eb47-47e9-8c6b-ea52b9d70190",
                    "location": {
                        "position": {
                            "latitude": 37.80841,
                            "longitude": -122.410097
                        }
                    },
                    "position": {
                        "latitude": 37.80841,
                        "longitude": -122.410097
                    },
                    "driveToLocation": {}
                },
                {
                    "id": "be8ed79b-ad50-4597-9d5f-8662db4d8316",
                    "tripId":

## ride-hail-driver: CompleteSteps

Driving to second dropoff.

In [27]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now only have the dropoff left since the drive to second dropoff step has been completed.

In [28]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

step_id = get_vehicle_state_response['state']['plan']['step'][0]['id']
trip_id = get_vehicle_state_response['state']['plan']['step'][0]['tripId']

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": [
                {
                    "id": "be8ed79b-ad50-4597-9d5f-8662db4d8316",
                    "tripId": "9500ca86-eb47-47e9-8c6b-ea52b9d70190",
                    "location": {
                        "position": {
                            "latitude": 37.80841,
                            "longitude": -122.410097
                        }
                    },
                    "position": {
                        "latitude": 37.80841,
                        "longitude": -122.410097
                    },
                    "dropoffRider": {
                        "riderCount": 2,
                        "riderInfo": {
                            "contactInfo": {
           

## ride-hail-driver: CompleteSteps

Second dropoff.

In [29]:
COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

complete_step_request = {
    "vehicle_id": vehicle_id,
    "trip_id": trip_id,
    "steps_to_complete": [{
        "step_id": step_id
    }]
}

response = requests.post(
    COMPLETE_STEP_URL,
    headers=AUTHORIZATION_HEADER,
    json=complete_step_request
)
response.raise_for_status()

complete_step_response = response.json()

print("Got successful complete step response:")
print(json.dumps(complete_step_response, indent=4))

Got successful complete step response:
{}


## ride-hail-driver: GetVehicleState

Checking on the state of the vehicle. The plan will now be empty since the second dropoff step has been completed.

In [30]:
GET_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/GetVehicleState"

get_vehicle_state_request = {
    "id": vehicle_id
}

response = requests.post(
    GET_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_vehicle_state_request
)
response.raise_for_status()

get_vehicle_state_response = response.json()

print("Got successful get trip state response:")
print(json.dumps(get_vehicle_state_response, indent=4))

Got successful get trip state response:
{
    "state": {
        "position": {
            "latitude": 37.78898783770332,
            "longitude": -122.38959319031967
        },
        "heading": 0.0,
        "lastActiveTime": "2020-04-22T17:09:26.535Z",
        "plan": {
            "step": []
        },
        "readiness": true,
        "isReachable": true
    }
}


## ride-hail-driver: RemoveVehicle

Removing a vehicle from a fleet using ride-hail-driver. 

In [31]:
REMOVE_VEHICLE_URL = "https://api.rideos.ai/ride-hail-driver/v1/RemoveVehicle"

remove_vehicle_request = {
    "id": vehicle_id
}

response = requests.post(
    REMOVE_VEHICLE_URL,
    headers=AUTHORIZATION_HEADER,
    json=remove_vehicle_request
)
response.raise_for_status()

remove_vehicle_response = response.json()

print("Got successful remove vehicle response:")
print(json.dumps(remove_vehicle_response, indent=4))

Got successful remove vehicle response:
{}


## ride-hail-operations: RemoveFleet

Removing the fleet using ride-hail-operations.

In [32]:
REMOVE_FLEET_URL = "https://api.rideos.ai/ride-hail-operations/v1/RemoveFleet"

remove_fleet_request = {
    "id": fleet_id
}

response = requests.post(
    REMOVE_FLEET_URL,
    headers=AUTHORIZATION_HEADER,
    json=remove_fleet_request
)
response.raise_for_status()

remove_fleet_response = response.json()

print("Got successful remove fleet response:")
print(json.dumps(remove_fleet_response, indent=4))

Got successful remove fleet response:
{}
